In [1]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam, SGD
from torchsummary import summary
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
waistDF = pickle.load( open( "waist_final_10272800_new.pkl", "rb" ))
wristDF = pickle.load( open( "wrist_final_10272800_new.pkl", "rb" ))
#otherDF = pickle.load( open( "others.pkl", "rb" ))
wristdrivingDF = pickle.load( open( "equal_wrist_dirving_final_4800000.pkl", "rb" ) )
waistdrivingDF = pickle.load( open( "equal_waist_dirving_final_4800000.pkl", "rb" ) )
wristothersDF = pickle.load( open( "equal_wrist_others_final_10220000.pkl", "rb" ) )
waistothersDF = pickle.load( open( "equal_waist_others_final_10220000.pkl", "rb" ) )

# </h3>Dataset class</h3>


In [3]:
from torch.utils import data


class Dataset(data.Dataset):
   
    def __init__(self, X1,X2,Y):
        'Initialization'
        self.X1 = X1
        self.X2 = X2
        self.Y = Y

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.X1)
   
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        x1 = self.X1[index]
        x2 = self.X2[index]
        y = self.Y[index]

        return x1,x2,y

<h3>Extracting X train and Y train</h3>

In [4]:
XWrist = wristDF[['x','y','z']]
YWrist = wristDF[['walk']]
XWaist = waistDF[['x','y','z']]
YWaist = waistDF[['walk']]

XWaistDriving = waistdrivingDF[['x','y','z']]
YWaistDriving = waistdrivingDF[['walk']]
XWristDriving = wristdrivingDF[['x','y','z']]
YWristDriving = wristdrivingDF[['walk']]

XWaistothers = waistothersDF[['x','y','z']]
YWaistothers = waistothersDF[['walk']]
XWristothers = wristothersDF[['x','y','z']]
YWristothers = wristothersDF[['walk']]

In [5]:
len(XWaistDriving)

10140000

In [6]:
###preprocessing data -1 to 1 ###
XWrist = preprocessing.normalize(XWrist.values)
#YWrist = wristDF[['walk']]
XWaist = preprocessing.normalize(XWaist.values)
#YWaist = waistDF[['walk']]

XWaistDriving = preprocessing.normalize(XWaistDriving.values)
#YWaistDriving = waistdrivingDF[['walk']]
XWristDriving = preprocessing.normalize(XWristDriving.values)
#YWristDriving = wristdrivingDF[['walk']]

XWaistothers = preprocessing.normalize(XWaistothers.values)
#YWaistothers = waistothersDF[['walk']]
XWristothers = preprocessing.normalize(XWristothers.values)
#YWristothers = wristothersDF[['walk']]

# <h3>Reshapping the data</h3>

In [7]:
#X_train = XLabel.values.reshape(98918,100,3)
#Y_train = YLabel.values.reshape(98918,100,1)
XWrist_train = XWrist.reshape(102728,100,3)
YWrist_train = np.full((102728,100,3),[1,0,0])
XWaist_train = XWaist.reshape(102728,100,3)
YWaist_train = np.full((102728,100,3),[1,0,0])
XWristDriving_train = XWristDriving.reshape(101400,100,3)
YWristDriving_train = np.full((101400,100,3),[0,1,0])
XWaistDriving_train = XWaistDriving.reshape(101400,100,3)
YWaistDriving_train = np.full((101400,100,3),[0,1,0])
XWristothers_train = XWristothers.reshape(102200,100,3)
YWristothers_train = np.full((102200,100,3),[0,0,1])
XWaistothers_train = XWaistothers.reshape(102200,100,3)
YWaistothers_train = np.full((102200,100,3),[0,0,1])



In [8]:
len(YWaistothers_train[1])

100

In [9]:
X_train_waist = np.concatenate([XWrist_train,XWristDriving_train,XWristothers_train])
Y_train_waist = np.concatenate([YWrist_train,YWristDriving_train,YWristothers_train])
X_train_wrist = np.concatenate([XWaist_train,XWaistDriving_train,XWaistothers_train])
Y_train_wrist = np.concatenate([YWaist_train,YWaistDriving_train,YWaistothers_train])

In [10]:
X_train_waist.shape

(306328, 100, 3)

In [11]:
Y_train_waist.shape

(306328, 100, 3)

In [12]:
X_train_wrist.shape

(306328, 100, 3)

In [13]:
Y_train_wrist.shape

(306328, 100, 3)

In [14]:
len(X_train_waist)

306328

In [15]:
X_train_combine=[]

In [16]:
for i in range (len(X_train_waist)):
    temp = []
    temp.append(X_train_waist[i])
    temp.append(X_train_wrist[i])
    X_train_combine.append(temp)

In [17]:
X_train_combine = np.array(X_train_combine)

In [18]:
Y_train_combine=[]

In [19]:
for i in range (len(Y_train_waist)):
    temp = []
    temp.append(Y_train_waist[i])
    temp.append(Y_train_wrist[i])
    Y_train_combine.append(temp)

In [20]:
Y_train_combine = np.array(Y_train_combine)

In [21]:
print(X_train_waist.shape)
print(Y_train_waist.shape)
print(Y_train_combine.shape)

(306328, 100, 3)
(306328, 100, 3)
(306328, 2, 100, 3)


# <h3>Training, validation and test split</h3>


In [22]:
combine_train_x, combine_val_x, combine_y, combine_val_y = train_test_split(X_train_combine, Y_train_combine, test_size = 0.3,train_size=0.7)
print(combine_val_x.shape,combine_val_y.shape)
combine_test_x, combine_val_x, combine_test_y, combine_val_y = train_test_split(combine_val_x, combine_val_y, test_size = 2/3,train_size=1/3)

(91899, 2, 100, 3) (91899, 2, 100, 3)


In [23]:
print(combine_train_x.shape,combine_y.shape)
print(combine_val_x.shape,combine_val_y.shape)
print(combine_test_x.shape,combine_test_y.shape)

(214429, 2, 100, 3) (214429, 2, 100, 3)
(61266, 2, 100, 3) (61266, 2, 100, 3)
(30633, 2, 100, 3) (30633, 2, 100, 3)


In [24]:
waist_train_x=[] 
waist_train_y=[]
waist_val_x=[]
waist_val_y=[]
waist_test_x=[]
waist_test_y=[]
wrist_train_x=[] 
wrist_train_y=[]
wrist_val_x=[]
wrist_val_y=[]
wrist_test_x=[]
wrist_test_y=[]

In [25]:
for i in range (len(combine_train_x)):
    #temp = []
    waist_train_x.append(combine_train_x[i][0])
    wrist_train_x.append(combine_train_x[i][1])
    
for i in range (len(combine_y)):
    #temp = []
    waist_train_y.append(combine_y[i][0])
    wrist_train_y.append(combine_y[i][1])
    
    
for i in range (len(combine_val_x)):
    #temp = []
    waist_val_x.append(combine_val_x[i][0])
    wrist_val_x.append(combine_val_x[i][1])
    
for i in range (len(combine_val_y)):
    #temp = []
    waist_val_y.append(combine_val_y[i][0])
    wrist_val_y.append(combine_val_y[i][1])
    
    
for i in range (len(combine_test_x)):
    #temp = []
    waist_test_x.append(combine_test_x[i][0])
    wrist_test_x.append(combine_test_x[i][1])
    
for i in range (len(combine_test_y)):
    #temp = []
    waist_test_y.append(combine_test_y[i][0])
    wrist_test_y.append(combine_test_y[i][1])

In [26]:
waist_train_x=np.array(waist_train_x)
waist_train_y=np.array(waist_train_y)
waist_val_x=np.array(waist_val_x)
waist_val_y=np.array(waist_val_y)
waist_test_x=np.array(waist_test_x)
waist_test_y=np.array(waist_test_y)
wrist_train_x=np.array(wrist_train_x)
wrist_train_y=np.array(wrist_train_y)
wrist_val_x=np.array(wrist_val_x)
wrist_val_y=np.array(wrist_val_y)
wrist_test_x=np.array(wrist_test_x)
wrist_test_y=np.array(wrist_test_y)

In [27]:
print(waist_train_x.shape,waist_train_y.shape)
print(waist_val_x.shape,waist_val_y.shape)
print(waist_test_x.shape,waist_test_y.shape)

(214429, 100, 3) (214429, 100, 3)
(61266, 100, 3) (61266, 100, 3)
(30633, 100, 3) (30633, 100, 3)


In [28]:
print(wrist_train_x.shape,wrist_train_y.shape)
print(wrist_val_x.shape,wrist_val_y.shape)
print(wrist_test_x.shape,wrist_test_y.shape)

(214429, 100, 3) (214429, 100, 3)
(61266, 100, 3) (61266, 100, 3)
(30633, 100, 3) (30633, 100, 3)


In [29]:
#X_trainWaist[80]

In [30]:
X_trainWaist = waist_train_x.reshape(214429,1,100,3)
Y_trainWaist = waist_train_y.reshape(214429,100,3)
temp=[]
for z in Y_trainWaist:
    temp.append(z[0])
Y_trainWaist = np.array(temp)


X_valWaist = waist_val_x.reshape(61266,1,100,3)
Y_valWaist = waist_val_y.reshape(61266,100,3)
temp=[]
for z in Y_valWaist:
    temp.append(z[0])
Y_valWaist = np.array(temp)



#X_test = test_x.reshape(97459,1,100,3)
#Y_test = test_y.reshape(97459,100)
X_testWaist = waist_test_x.reshape(30633,1,100,3)
Y_testWaist = waist_test_y.reshape(30633,100,3)
temp=[]
for z in Y_testWaist:
    temp.append(z[0])
Y_testWaist = np.array(temp)



In [31]:
print(X_trainWaist.shape,Y_trainWaist.shape)
print(X_valWaist.shape,Y_valWaist.shape)
print(X_testWaist.shape,Y_testWaist.shape)

(214429, 1, 100, 3) (214429, 3)
(61266, 1, 100, 3) (61266, 3)
(30633, 1, 100, 3) (30633, 3)


In [32]:
X_trainWrist = wrist_train_x.reshape(214429,1,100,3)
Y_trainWrist = wrist_train_y.reshape(214429,100,3)
temp=[]
for z in Y_trainWrist:
    temp.append(z[0])
Y_trainWrist = np.array(temp)


X_valWrist = wrist_val_x.reshape(61266,1,100,3)
Y_valWrist = wrist_val_y.reshape(61266,100,3)
temp=[]
for z in Y_valWrist:
    temp.append(z[0])
Y_valWrist = np.array(temp)



#X_test = test_x.reshape(97459,1,100,3)
#Y_test = test_y.reshape(97459,100)
X_testWrist = waist_test_x.reshape(30633,1,100,3)
Y_testWrist = waist_test_y.reshape(30633,100,3)
temp=[]
for z in Y_testWrist:
    temp.append(z[0])
Y_testWrist = np.array(temp)



In [33]:
Y_trainWrist[0]

array([0, 0, 1])

In [34]:
Y_trainWaist

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [35]:
print(X_trainWrist.shape,Y_trainWrist.shape)
print(X_valWrist.shape,Y_valWrist.shape)
print(X_testWrist.shape,Y_testWrist.shape)

(214429, 1, 100, 3) (214429, 3)
(61266, 1, 100, 3) (61266, 3)
(30633, 1, 100, 3) (30633, 3)


In [36]:
np.save('splitdata/X_trainWrist_equal5', X_trainWrist) 
np.save('splitdata/Y_train_equal5', Y_trainWrist) 
np.save('splitdata/X_valWrist_equal5', X_valWrist) 
np.save('splitdata/Y_valWrist_equal5', Y_valWrist) 
np.save('splitdata/X_testWrist_equal5', X_testWrist) 
np.save('splitdata/Y_testWrist_equal5', Y_testWrist) 


np.save('splitdata/X_trainWaist_equal5', X_trainWaist) 
np.save('splitdata/Y_trainWaist_equal5', Y_trainWaist) 
np.save('splitdata/X_valWaist_equal5', X_valWaist) 
np.save('splitdata/Y_valWaist_equal5', Y_valWaist) 
np.save('splitdata/X_testWaist_equal5', X_testWaist) 
np.save('splitdata/Y_testWaist_equal5', Y_testWaist) 




FileNotFoundError: [Errno 2] No such file or directory: 'splitdata/X_trainWrist_equal5.npy'

# <h3>Tensors</h3>

In [ ]:
X_trainWaist = torch.from_numpy(X_trainWaist)
Y_trainWaist = torch.from_numpy(Y_trainWaist)

In [ ]:
X_trainWrist = torch.from_numpy(X_trainWrist)
Y_trainWrist = torch.from_numpy(Y_trainWrist)

In [ ]:
print(X_trainWaist.shape,Y_trainWaist.shape)

In [ ]:
print(X_trainWrist.shape,Y_trainWrist.shape)

In [ ]:
X_valWaist = torch.from_numpy(X_valWaist)
Y_valWaist = torch.from_numpy(Y_valWaist)

In [ ]:
print(X_valWaist.shape,Y_valWaist.shape)

In [ ]:
X_valWrist = torch.from_numpy(X_valWrist)
Y_valWrist = torch.from_numpy(Y_valWrist)

In [ ]:
print(X_valWrist.shape,Y_valWrist.shape)

In [ ]:
X_testWaist = torch.from_numpy(X_testWaist)
Y_testWaist = torch.from_numpy(Y_testWaist)

In [ ]:
print(X_testWaist.shape,Y_testWaist.shape)

In [ ]:
X_testWrist = torch.from_numpy(X_testWrist)
Y_testWrist = torch.from_numpy(Y_testWrist)

In [37]:
print(X_testWrist.shape,Y_testWrist.shape)

(30633, 1, 100, 3) (30633, 3)


In [38]:
print(X_trainWaist.shape)
print(Y_trainWaist.shape)

(214429, 1, 100, 3)
(214429, 3)


In [39]:
import torch
from torch.utils import data

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Parameters
params = {'batch_size': 256,'num_workers': 6}
#params = {'batch_size': 256,'num_workers': 6}
max_epochs = 10


# Generators
training_setCombine = Dataset(X_trainWaist, X_trainWrist, Y_trainWaist)
training_generatorCombine = data.DataLoader(training_setCombine, **params)

validation_setCombine = Dataset(X_valWaist,X_valWrist, Y_valWaist)
validation_generatorCombine = data.DataLoader(validation_setCombine, **params)

test_set = Dataset(X_testWaist,X_testWrist, Y_testWaist)
testset_generatorCombine = data.DataLoader(test_set, **params)

In [40]:
training_setCombine.__getitem__(8)[2]

array([0, 0, 1])

In [41]:
a=training_setCombine.__getitem__(8)[2]

In [42]:
a

array([0, 0, 1])

# <h3>Building model</h3>

In [43]:
class ConvNetWaist(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNetWaist, self).__init__()
        
        
        self.fc1 = nn.Linear(100*2*4,64)
        self.fc2 = nn.Linear(64,2)
        self.fc1 = nn.Linear(100*2*4,64)
        self.fc2 = nn.Linear(64,2)
       
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        #out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        #out = F.relu(self.fc1(out))
        #out = F.relu(self.fc1(out))
        #out = self.fc2(out)
        return out

In [ ]:
class ConvNetWrist(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvNetWrist, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=(3,3), stride=1,padding=1),
            nn.BatchNorm2d(128),
            #nn.Dropout(p=0.9),
            nn.ReLU()
            )
            
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=(3,3), stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.8),
            nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2, stride=2,padding=1))
            nn.MaxPool2d(kernel_size=2, stride=2,padding=1))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=(3,3), stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.Dropout(p=0.8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        #self.fc1 = nn.Linear(100*2*4,64)
        #self.fc2 = nn.Linear(64,2)
        #self.fc1 = nn.Linear(100*2*4,64)
        #self.fc2 = nn.Linear(64,2)
       
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        #out = F.relu(self.fc1(out))
        #out = F.relu(self.fc1(out))
        #out = self.fc2(out)
        return out

In [ ]:
class Combine_Waist_Wrist(nn.Module):
    def __init__(self, modelA, modelB):
        super(Combine_Waist_Wrist, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        
        self.fc1 = nn.Linear(100*2*4*2,64)
        #self.fc1 = nn.Linear(100*2*2,64)
        
        self.classifier = nn.Linear(64,3)   ### 2 -> 1 , make labels 1 dimensions: 0 or 1
        #self.softmax = nn.Softmax(dim=3)
        #linear layers
        #linear layers
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        out = torch.cat((x1, x2), dim=1)
        
        out = F.relu(self.fc1(out))
        #out = (self.fc1(out))
        x = self.classifier(out)  # not relu
        #x = F.tanh(out)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
from torch.optim.lr_scheduler import StepLR
modelWaist = ConvNetWaist()
modelWrist = ConvNetWrist()
modelCombine = Combine_Waist_Wrist(modelWaist,modelWrist)
#model=model.Dropout(p=0.2)
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.MultiLabelSoftMarginLoss()
#criterion = nn.CrossEntropyLoss()
#optimizerWaist = Adam(modelWaist.parameters(), lr=0.02)
#optimizerWrist = Adam(modelWrist.parameters(), lr=0.02)
#optimizerCombine = optimizerWaist +optimizerWrist

params = list(modelWaist.parameters()) + list(modelWrist.parameters())
optimizerCombine = Adam(params, lr=0.0001)
scheduler = StepLR(optimizerCombine, step_size=1, gamma=0.9)
#weight_decay=1e-5

#optimizerCombine = SGD(modelCombine.parameters(), lr=0.001)
#optimizer = SGD(model.parameters(), lr=0.01)
#learning_rate = 0.01


if torch.cuda.is_available():
    #model = nn.DataParallel(model)
    modelWaist = modelWaist.cuda()
    modelWrist = modelWrist.cuda()
    modelCombine = modelCombine.cuda()
    criterion = criterion.cuda()

modelWaist.to(device)
modelWrist.to(device)
modelCombine.to(device)



In [ ]:
summary(modelCombine, [(1,100, 3),(1,100, 3)])

In [ ]:
##### how it will look like with new dataloader

n_epochs = 50
# empty list to store training losses
#train_losses_waist = []
#train_losses_wrist = []
train_losses_combine = []
train_losses_combine_avg = []

# empty list to store validation losses
#val_losses_waist = []
#val_losses_wrist = []
val_losses_combine = []
val_losses_combine_avg = []


def train():
    modelCombine.train()
    tr_loss = 0
    
    #n_epochs = 10
    best_loss=99999999999999
    for epoch in range(n_epochs):
        tr_loss = 0
        val_loss = 0
        #scheduler.step()
        temp1=[]
        temp2=[]
        for local_batch_waist, local_batch_wrist, local_labels in training_generatorCombine:
            local_batch_waist, local_batch_wrist, local_labels = local_batch_waist.to(device), local_batch_wrist.to(device), local_labels.to(device)
            
            optimizerCombine.zero_grad()
            #output_train = modelCombine(local_batch_waist.float(), local_batch_wrist.float())
            output_train = modelCombine(local_batch_waist.float(), local_batch_wrist.float())
            
            #print(output_train.size())
            #print(local_labels.float().size())
            #print(output_train)
            #print(torch.argmax(output_train, 1))
            #print(torch.argmax(local_labels.float(), 1))
            #print(local_labels.float().squeeze().size())
            ##print(output_train)
            #print(local_labels.float.uniform_(0, 1))
            #loss_train = criterion(torch.argmax(output_train, 1), torch.argmax(local_labels.float(), 1))
            loss_train = criterion(output_train, local_labels.squeeze(1).float())
            
            #print(loss_train)

            temp1.append(loss_train.item())
            train_losses_combine.append(loss_train.item())
            
            loss_train.backward()
            optimizerCombine.step()
            #tr_loss = loss_train.item()
            #tr_loss = tr_loss + loss_train.item()
        avg_loss = np.mean(temp1)
        train_losses_combine_avg.append(avg_loss)
        
        print(tr_loss)
        print('Epoch : ',epoch+1, '\t', 'loss :', avg_loss)
        #print('Epoch : ',epoch+1, '\t', 'loss :', tr_loss)
        
        
        with torch.set_grad_enabled(False):
            for local_batch_waist, local_batch_wrist, local_labels in validation_generatorCombine:
                local_batch_waist, local_batch_wrist, local_labels = local_batch_waist.to(device), local_batch_wrist.to(device), local_labels.to(device)
                
                
                
                output_val = modelCombine(local_batch_waist.float(), local_batch_wrist.float())
                #print(output_val.size())
                #print(local_labels.float().size())
                #print(output_val.unsqueeze(1).size())
                #print(local_labels.float().unsqueeze(1).size())
                
                loss_val = criterion(output_val, local_labels.float())
                val_losses_combine.append(loss_val.item())
                temp2.append(loss_val.item())
                
                #val_loss = val_loss + loss_val
            avg_loss = np.mean(temp2)
            if (avg_loss<=best_loss):
                #torch.save(model.state_dict(), best_model_path)
                torch.save(modelCombine.state_dict(), "seventh_equal_best.pt")
                best_loss=avg_loss
                print("best:  ",avg_loss)
                # safe the best mode -> lowest validation loss
                
            
            val_losses_combine_avg.append(avg_loss)
            print('Epoch : ',epoch+1, '\t', 'loss :', avg_loss)
            
            

In [ ]:
train()

In [ ]:
torch.save(modelCombine.state_dict(), "seventh_equal_last.pt")

In [ ]:
#torch.save(modelCombine.state_dict(), "Second_equal_last.pt")

In [ ]:
#modelC = init_protonet()
'''
modelWaist1 = ConvNetWaist()#
modelWrist1 = ConvNetWrist()
modelCombine1 = Combine_Waist_Wrist(modelWaist1,modelWrist1)
modelWaist1.to(device)
modelWrist1.to(device)
modelCombine1.to(device)
modelCombine1.load_state_dict(torch.load("Second_equal_last.pt"))
'''

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses_combine_avg, label='Training loss')
plt.plot(val_losses_combine_avg, label='Validation loss')
plt.legend()
plt.show()
plt.savefig('Equal7_1.png')

In [ ]:
plt.plot(train_losses_combine, label='Training loss')
plt.plot(val_losses_combine, label='Validation loss')
plt.legend()
plt.show()
plt.savefig('Equal7.png')

In [ ]:
#modelCombine = torch.load("1Combine_step_model_dropout0.9_weightdecay_scheduler_epoch20_equal.pt")
#modelCombine.eval()

In [ ]:
with torch.no_grad():
    for local_batch_waist, local_batch_wrist, local_labels in testset_generatorCombine:
        print(local_labels)
        break

In [ ]:
# prediction for training set
output =[]
prediction=[]
count=0
with torch.no_grad():
    for local_batch_waist, local_batch_wrist, local_labels in testset_generatorCombine:
        count=count+1
            #local_batch_waist, local_batch_wrist, local_labels = local_batch_waist.to(device), local_batch_wrist.to(device), local_labels.to(device)
        a=modelCombine(local_batch_waist.float().cuda(), local_batch_wrist.float().cuda())
        #print(a)
        output.append(a)
        #output.append(modelWaist(local_batch.float().cuda()))
        prediction.append(local_labels)
        
    print(count)        
"""    
temp =[]
for x in output:
    softmax = torch.exp(x).cpu()
    prob = list(softmax.numpy())
    temp.extend(prob)
predictions3 = np.argmax(temp, axis=1)
"""

In [ ]:
len(output)

In [ ]:
loss1 = open("splitdata/outputequatest6_1.pkl", 'wb')
pickle.dump(output, loss1)
loss1.close()
loss2 = open("splitdata/predictionequaltest6_1.pkl", 'wb')
pickle.dump(prediction, loss2)
loss2.close()

In [ ]:
count_0=0
count_1=0
count_2=0
rcount_0=0
rcount_1=0
rcount_2=0
should1_but0=0
should1_but2=0
should2_but0=0
should2_but1=0
should0_but1=0
should0_but2=0

for i in range(0,(len(output))):
    pred=output[i]
    labelset=prediction[i]
    #print(i)
    for j in range(0,len(pred)):
        if(torch.argmax(labelset[j].cpu()).item()==0):
            count_0=count_0+1
            if(torch.argmax(pred[j].cpu()).item()==0):
                rcount_0=rcount_0+1   
            if(torch.argmax(pred[j].cpu()).item()==1):
                should0_but1=should0_but1+1
            if(torch.argmax(pred[j].cpu()).item()==2):
                should0_but2=should0_but2+1
        #else:
            
        elif(torch.argmax(labelset[j].cpu()).item()==1):
            count_1=count_1+1
            if(torch.argmax(pred[j].cpu()).item()==1):
                rcount_1=rcount_1+1
            if(torch.argmax(pred[j].cpu()).item()==0):
                should1_but0=should1_but0+1
            if(torch.argmax(pred[j].cpu()).item()==2):
                should1_but2=should1_but2+1

        elif(torch.argmax(labelset[j].cpu()).item()==2):
            count_2=count_2+1
            if(torch.argmax(pred[j].cpu()).item()==2):
                rcount_2=rcount_2+1
            if(torch.argmax(pred[j].cpu()).item()==0):
                should2_but0=should2_but0+1
            if(torch.argmax(pred[j].cpu()).item()==1):
                should2_but1=should2_but1+1    
        #if torch.argmax(output[j].cpu()).item() == torch.argmax(prediction[j].cpu()).item() ==0

In [ ]:
print(count_0,rcount_0)
print(count_1,rcount_1)
print(count_2,rcount_2)
print(should2_but0)
print(should2_but1)
print(should1_but0)
print(should1_but2)
print(should0_but1)
print(should0_but2)